In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import matplotlib.gridspec as gridspec

np.random.seed(42)
plt.rcParams['figure.figsize'] = [12, 8]

def root_raised_cosine(alpha, span, sps):
    t = np.arange(-span*sps, span*sps + 1) / sps
    pulse = np.zeros_like(t)
    
    for i, t_val in enumerate(t):
        if t_val == 0:
            pulse[i] = 1.0 - alpha + (4 * alpha / np.pi)
        elif alpha != 0 and t_val == 1/(4*alpha):
            pulse[i] = (alpha/np.sqrt(2)) * ((1+2/np.pi)*np.sin(np.pi/(4*alpha)) + 
                                           (1-2/np.pi)*np.cos(np.pi/(4*alpha)))
        elif alpha != 0 and t_val == -1/(4*alpha):
            pulse[i] = (alpha/np.sqrt(2)) * ((1+2/np.pi)*np.sin(np.pi/(4*alpha)) + 
                                           (1-2/np.pi)*np.cos(np.pi/(4*alpha)))
        else:
            numerator = np.sin(np.pi*t_val*(1-alpha)) + 4*alpha*t_val*np.cos(np.pi*t_val*(1+alpha))
            denominator = np.pi*t_val*(1-(4*alpha*t_val)**2)
            pulse[i] = numerator / denominator
    
    # Нормализация
    pulse = pulse / np.sqrt(np.sum(pulse**2))
    return t, pulse

def generate_source_data(num_bits=100):
    """Генерация исходных битовых данных"""
    print("=== ШАГ 1: ГЕНЕРАЦИЯ ИСХОДНЫХ ДАННЫХ ===")
    
    # Генерация случайных битов
    source_bits = np.random.randint(0, 2, num_bits)
    print(f"Сгенерировано битов: {len(source_bits)}")
    print(f"Первые 20 битов: {source_bits[:20]}")
    
    return source_bits

def bits_to_qpsk_symbols(bits):
    """Преобразование битов в QPSK символы"""
    if len(bits) % 2 != 0:
        bits = np.append(bits, 0)  # Добавляем нуль если нечетное количество
    
    # Разделение на I и Q компоненты
    i_bits = bits[0::2]  # Четные биты -> I
    q_bits = bits[1::2]  # Нечетные биты -> Q
    
    # Преобразование в символы: 0 -> -1, 1 -> +1
    i_symbols = 2 * i_bits - 1
    q_symbols = 2 * q_bits - 1
    
    # Комплексные символы
    symbols = i_symbols + 1j * q_symbols
    
    print(f"Преобразовано в {len(symbols)} QPSK символов")
    print(f"Биты I: {i_bits[:10]} -> Символы I: {i_symbols[:10]}")
    print(f"Биты Q: {q_bits[:10]} -> Символы Q: {q_symbols[:10]}")
    print(f"Первые 5 комплексных символов: {symbols[:5]}")
    
    return symbols, i_bits, q_bits, i_symbols, q_symbols

def create_pulse_shape(samples_per_symbol=16, beta=0.35, span=6):
    """Создание импульсной формы"""
    t, pulse = root_raised_cosine(beta, span, samples_per_symbol)
    return t, pulse

def shape_symbols(symbols, samples_per_symbol=16, span=6):
    """Формирование импульса символов"""
    # Апсемплинг (добавление нулей между символами)
    upsampled = np.zeros(len(symbols) * samples_per_symbol, dtype=complex)
    upsampled[::samples_per_symbol] = symbols
    
    # Создание импульсной формы
    t, pulse = create_pulse_shape(samples_per_symbol, span=span)
    
    # Свертка с импульсной формой
    shaped_signal = signal.convolve(upsampled, pulse, mode='same')
    
    # Удаление краевых эффектов
    start_idx = span * samples_per_symbol
    end_idx = -span * samples_per_symbol if span * samples_per_symbol > 0 else None
    shaped_signal = shaped_signal[start_idx:end_idx]
    
    print(f"Апсемплинг: {len(symbols)} символов -> {len(upsampled)} отсчетов")
    print(f"После формирования импульса: {len(shaped_signal)} отсчетов")
    
    return shaped_signal, upsampled

# Генерация и визуализация исходных данных
source_bits = generate_source_data(200)
symbols, i_bits, q_bits, i_symbols, q_symbols = bits_to_qpsk_symbols(source_bits)
shaped_signal, upsampled = shape_symbols(symbols)

# Визуализация генерации сигнала
fig = plt.figure(figsize=(16, 12))
gs = gridspec.GridSpec(3, 3)

# Исходные биты
ax1 = plt.subplot(gs[0, 0])
ax1.stem(range(20), source_bits[:20], basefmt=" ")
ax1.set_title('Исходные биты (первые 20)')
ax1.set_xlabel('Индекс бита')
ax1.set_ylabel('Значение')
ax1.grid(True)

# Созвездие QPSK
ax2 = plt.subplot(gs[0, 1])
constellation_points = [1+1j, 1-1j, -1+1j, -1-1j]
colors = ['red', 'blue', 'green', 'purple']
labels = ['00', '01', '10', '11']

for point, color, label in zip(constellation_points, colors, labels):
    ax2.scatter(np.real(point), np.imag(point), c=color, s=100, label=label)
    ax2.text(np.real(point)+0.1, np.imag(point)+0.1, label, fontsize=12)

ax2.set_title('Созвездие QPSK')
ax2.set_xlabel('I компонента')
ax2.set_ylabel('Q компонента')
ax2.legend()
ax2.grid(True)
ax2.axis('equal')

# Переданные символы
ax3 = plt.subplot(gs[0, 2])
ax3.scatter(np.real(symbols[:50]), np.imag(symbols[:50]), alpha=0.6, s=50)
ax3.set_title('Переданные символы (первые 50)')
ax3.set_xlabel('I')
ax3.set_ylabel('Q')
ax3.grid(True)

# I и Q компоненты до формирования
ax4 = plt.subplot(gs[1, 0])
time_symbols = np.arange(len(i_symbols[:20]))
ax4.stem(time_symbols, i_symbols[:20], linefmt='b-', markerfmt='bo', 
         basefmt=' ', label='I компонента')
ax4.stem(time_symbols, q_symbols[:20], linefmt='r--', markerfmt='rx', 
         basefmt=' ', label='Q компонента')
ax4.set_title('I и Q символы (первые 20)')
ax4.set_xlabel('Индекс символа')
ax4.set_ylabel('Амплитуда')
ax4.legend()
ax4.grid(True)

# Импульсная форма
ax5 = plt.subplot(gs[1, 1])
t, pulse = create_pulse_shape()
ax5.plot(t, pulse, 'g-', linewidth=2)
ax5.set_title('Импульсная форма\n(корень из приподнятого косинуса)')
ax5.set_xlabel('Отсчеты')
ax5.set_ylabel('Амплитуда')
ax5.grid(True)

# Сформированный сигнал
ax6 = plt.subplot(gs[1, 2])
time_shaped = np.arange(200)
ax6.plot(time_shaped, np.real(shaped_signal)[:200], 'b-', label='I компонента', alpha=0.8)
ax6.plot(time_shaped, np.imag(shaped_signal)[:200], 'r-', label='Q компонента', alpha=0.8)
ax6.set_title('Сформированный сигнал (первые 200 отсчетов)')
ax6.set_xlabel('Отсчеты')
ax6.set_ylabel('Амплитуда')
ax6.legend()
ax6.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def modulate_carrier(shaped_signal, carrier_freq=0.05, sample_rate=1.0):
    """Модуляция на несущую частоту"""
    print("\n=== ШАГ 2: МОДУЛЯЦИЯ НА НЕСУЩУЮ ===")
    
    t = np.arange(len(shaped_signal)) / sample_rate
    
    # Разделение на I и Q компоненты
    i_signal = np.real(shaped_signal)
    q_signal = np.imag(shaped_signal)
    
    # Квадратурная модуляция
    carrier_i = i_signal * np.cos(2 * np.pi * carrier_freq * t)
    carrier_q = q_signal * np.sin(2 * np.pi * carrier_freq * t)
    
    # Суммирование для получения вещественного сигнала
    modulated_signal = carrier_i - carrier_q  # Минус для правильной фазы
    
    print(f"Несущая частота: {carrier_freq:.3f} (относительная)")
    print(f"Длина модулированного сигнала: {len(modulated_signal)} отсчетов")
    
    return modulated_signal, t, carrier_i, carrier_q

def add_channel_effects(signal, snr_db=20, frequency_offset=0.0, phase_offset=0.0):
    """Добавление эффектов канала связи"""
    # Добавление шума
    signal_power = np.mean(signal**2)
    noise_power = signal_power / (10**(snr_db/10))
    noise = np.sqrt(noise_power) * np.random.randn(len(signal))
    
    # Добавление частотного сдвига
    t = np.arange(len(signal))
    freq_shift = np.cos(2 * np.pi * frequency_offset * t)
    
    # Добавление фазового сдвига
    phase_shift = np.cos(phase_offset)
    
    impaired_signal = signal * freq_shift * phase_shift + noise
    
    print(f"Добавлен шум: SNR = {snr_db} dB")
    print(f"Частотный сдвиг: {frequency_offset:.4f}")
    print(f"Фазовый сдвиг: {phase_offset:.3f} рад")
    
    return impaired_signal, noise

# Модуляция и добавление эффектов канала
modulated, time_vec, carrier_i, carrier_q = modulate_carrier(shaped_signal)
received_signal, channel_noise = add_channel_effects(modulated, snr_db=15, 
                                                   frequency_offset=0.001, 
                                                   phase_offset=0.02)

# Визуализация модуляции
fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(3, 2)

# I и Q компоненты перед модуляцией
ax1 = plt.subplot(gs[0, 0])
ax1.plot(time_vec[:200], np.real(shaped_signal)[:200], 'b-', label='I компонента', linewidth=2)
ax1.plot(time_vec[:200], np.imag(shaped_signal)[:200], 'r-', label='Q компонента', linewidth=2)
ax1.set_title('I и Q компоненты до модуляции')
ax1.set_xlabel('Время')
ax1.set_ylabel('Амплитуда')
ax1.legend()
ax1.grid(True)

# Несущие колебания
ax2 = plt.subplot(gs[0, 1])
t_short = time_vec[:100]
ax2.plot(t_short, np.cos(2*np.pi*0.05*t_short), 'b-', label='cos(ωt) - для I', alpha=0.7)
ax2.plot(t_short, np.sin(2*np.pi*0.05*t_short), 'r-', label='sin(ωt) - для Q', alpha=0.7)
ax2.set_title('Несущие колебания')
ax2.set_xlabel('Время')
ax2.set_ylabel('Амплитуда')
ax2.legend()
ax2.grid(True)

# Модулированные компоненты
ax3 = plt.subplot(gs[1, 0])
ax3.plot(time_vec[:200], carrier_i[:200], 'b-', label='I × cos(ωt)', alpha=0.8)
ax3.plot(time_vec[:200], carrier_q[:200], 'r-', label='Q × sin(ωt)', alpha=0.8)
ax3.set_title('Модулированные I и Q компоненты')
ax3.set_xlabel('Время')
ax3.set_ylabel('Амплитуда')
ax3.legend()
ax3.grid(True)

# Полный модулированный сигнал
ax4 = plt.subplot(gs[1, 1])
ax4.plot(time_vec[:200], modulated[:200], 'g-', linewidth=2)
ax4.set_title('Полный модулированный сигнал\n(I×cos(ωt) - Q×sin(ωt))')
ax4.set_xlabel('Время')
ax4.set_ylabel('Амплитуда')
ax4.grid(True)

# Сигнал после канала
ax5 = plt.subplot(gs[2, 0])
ax5.plot(time_vec[:200], received_signal[:200], 'g-', label='Принятый сигнал', alpha=0.8)
ax5.plot(time_vec[:200], modulated[:200], 'k--', label='Идеальный сигнал', alpha=0.6)
ax5.set_title('Сигнал после канала связи')
ax5.set_xlabel('Время')
ax5.set_ylabel('Амплитуда')
ax5.legend()
ax5.grid(True)

# Шум канала
ax6 = plt.subplot(gs[2, 1])
ax6.plot(time_vec[:200], channel_noise[:200], 'm-', alpha=0.7)
ax6.set_title('Шум канала связи')
ax6.set_xlabel('Время')
ax6.set_ylabel('Амплитуда')
ax6.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def demodulate_signal(received_signal, carrier_freq=0.05, sample_rate=1.0):
    """Квадратурная демодуляция сигнала"""
    print("\n=== ШАГ 3: ДЕМОДУЛЯЦИЯ СИГНАЛА ===")
    
    t = np.arange(len(received_signal)) / sample_rate
    
    # Квадратурное детектирование
    i_demod = 2 * received_signal * np.cos(2 * np.pi * carrier_freq * t)
    q_demod = -2 * received_signal * np.sin(2 * np.pi * carrier_freq * t)  # Минус для компенсации
    
    print(f"Демодулированные I компонента: {len(i_demod)} отсчетов")
    print(f"Демодулированные Q компонента: {len(q_demod)} отсчетов")
    
    return i_demod, q_demod, t

def apply_matched_filter(i_signal, q_signal, samples_per_symbol=16, span=6):
    """Применение согласованного фильтра"""
    # Создание импульсной формы (такой же как на передаче)
    _, pulse = create_pulse_shape(samples_per_symbol, span=span)
    
    # Фильтрация I и Q компонент
    i_filtered = signal.convolve(i_signal, pulse, mode='same')
    q_filtered = signal.convolve(q_signal, pulse, mode='same')
    
    # Удаление краевых эффектов
    start_idx = span * samples_per_symbol
    end_idx = -span * samples_per_symbol if span * samples_per_symbol > 0 else None
    i_filtered = i_filtered[start_idx:end_idx]
    q_filtered = q_filtered[start_idx:end_idx]
    
    print(f"После согласованной фильтрации: {len(i_filtered)} отсчетов")
    
    return i_filtered, q_filtered

def timing_recovery(i_filtered, q_filtered, samples_per_symbol=16):
    """Тактовая синхронизация"""
    # Объединение I и Q для определения времени
    power_signal = i_filtered**2 + q_filtered**2
    
    # Поиск оптимальных точек отсчета (максимумы мощности)
    symbol_indices = []
    for i in range(samples_per_symbol//2, len(power_signal), samples_per_symbol):
        # Поиск максимума в окрестности предполагаемого времени символа
        start = max(0, i - samples_per_symbol//4)
        end = min(len(power_signal), i + samples_per_symbol//4)
        local_max_idx = start + np.argmax(power_signal[start:end])
        symbol_indices.append(local_max_idx)
    
    # Извлечение символов в оптимальные моменты времени
    i_symbols = i_filtered[symbol_indices]
    q_symbols = q_filtered[symbol_indices]
    recovered_symbols = i_symbols + 1j * q_symbols
    
    print(f"Восстановлено символов: {len(recovered_symbols)}")
    
    return recovered_symbols, symbol_indices, power_signal

# Демодуляция
i_demod, q_demod, t_demod = demodulate_signal(received_signal)
i_filtered, q_filtered = apply_matched_filter(i_demod, q_demod)
recovered_symbols, symbol_times, power_signal = timing_recovery(i_filtered, q_filtered)

# Визуализация демодуляции
fig = plt.figure(figsize=(16, 14))
gs = gridspec.GridSpec(4, 2)

# Демодулированные I и Q
ax1 = plt.subplot(gs[0, 0])
ax1.plot(t_demod[:300], i_demod[:300], 'b-', alpha=0.7, label='I демодулированная')
ax1.plot(t_demod[:300], q_demod[:300], 'r-', alpha=0.7, label='Q демодулированная')
ax1.set_title('Демодулированные I и Q компоненты')
ax1.set_xlabel('Время')
ax1.set_ylabel('Амплитуда')
ax1.legend()
ax1.grid(True)

# После согласованного фильтра
ax2 = plt.subplot(gs[0, 1])
ax2.plot(t_demod[:300], i_filtered[:300], 'b-', linewidth=2, label='I после фильтра')
ax2.plot(t_demod[:300], q_filtered[:300], 'r-', linewidth=2, label='Q после фильтра')
ax2.set_title('После согласованной фильтрации')
ax2.set_xlabel('Время')
ax2.set_ylabel('Амплитуда')
ax2.legend()
ax2.grid(True)

# Глазковая диаграмма для I компоненты
ax3 = plt.subplot(gs[1, 0])
samples_per_symbol = 16
for i in range(min(10, len(i_filtered)//samples_per_symbol)):
    start_idx = i * samples_per_symbol
    end_idx = start_idx + 2 * samples_per_symbol
    if end_idx < len(i_filtered):
        eye_segment = i_filtered[start_idx:end_idx]
        ax3.plot(np.arange(len(eye_segment)), eye_segment, 'b-', alpha=0.5)
ax3.set_title('Глазковая диаграмма (I компонента)')
ax3.set_xlabel('Отсчеты')
ax3.set_ylabel('Амплитуда')
ax3.grid(True)

# Восстановленное созвездие
ax5 = plt.subplot(gs[1, 1])
ax5.scatter(np.real(recovered_symbols), np.imag(recovered_symbols), 
           c='red', alpha=0.6, s=50, label='Восстановленные')
ax5.scatter(np.real(symbols[:len(recovered_symbols)]), 
           np.imag(symbols[:len(recovered_symbols)]), 
           c='blue', alpha=0.3, s=30, label='Исходные')
ax5.set_title('Сравнение созвездий')
ax5.set_xlabel('I компонента')
ax5.set_ylabel('Q компонента')
ax5.legend()
ax5.grid(True)

# Сравнение I компонент
ax6 = plt.subplot(gs[2, 0])
symbol_indices = range(min(20, len(recovered_symbols)))
ax6.stem(symbol_indices, np.real(symbols[:len(recovered_symbols)][:20]), 
         linefmt='b-', markerfmt='bo', basefmt=' ', label='Исходные I')
ax6.stem(symbol_indices, np.real(recovered_symbols[:20]), 
         linefmt='r--', markerfmt='rx', basefmt=' ', label='Восстановленные I')
ax6.set_title('Сравнение I компонент символов')
ax6.set_xlabel('Индекс символа')
ax6.set_ylabel('Амплитуда')
ax6.legend()
ax6.grid(True)

# Сравнение Q компонент
ax7 = plt.subplot(gs[2, 1])
ax7.stem(symbol_indices, np.imag(symbols[:len(recovered_symbols)][:20]), 
         linefmt='b-', markerfmt='bo', basefmt=' ', label='Исходные Q')
ax7.stem(symbol_indices, np.imag(recovered_symbols[:20]), 
         linefmt='r--', markerfmt='rx', basefmt=' ', label='Восстановленные Q')
ax7.set_title('Сравнение Q компонент символов')
ax7.set_xlabel('Индекс символа')
ax7.set_ylabel('Амплитуда')
ax7.legend()
ax7.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def symbols_to_bits(symbols, decision_threshold=0):
    """Преобразование символов обратно в биты"""
    # Принятие решения по символам
    i_decisions = (np.real(symbols) > decision_threshold).astype(int)
    q_decisions = (np.imag(symbols) > decision_threshold).astype(int)
    
    # Объединение битов
    recovered_bits = np.zeros(2 * len(i_decisions))
    recovered_bits[0::2] = i_decisions
    recovered_bits[1::2] = q_decisions
    
    print(f"Восстановлено битов: {len(recovered_bits)}")
    print(f"Первые 20 восстановленных битов: {recovered_bits[:20].astype(int)}")
    
    return recovered_bits, i_decisions, q_decisions

def calculate_performance(original_bits, recovered_bits, original_symbols, recovered_symbols):
    """Расчет характеристик системы"""
    # Bit Error Rate (BER)
    min_len = min(len(original_bits), len(recovered_bits))
    bit_errors = np.sum(original_bits[:min_len] != recovered_bits[:min_len])
    ber = bit_errors / min_len
    
    # Symbol Error Rate (SER)
    min_symbols = min(len(original_symbols), len(recovered_symbols))
    symbol_errors = 0
    for i in range(min_symbols):
        original_i = 1 if np.real(original_symbols[i]) > 0 else 0
        original_q = 1 if np.imag(original_symbols[i]) > 0 else 0
        recovered_i = 1 if np.real(recovered_symbols[i]) > 0 else 0
        recovered_q = 1 if np.imag(recovered_symbols[i]) > 0 else 0
        
        if original_i != recovered_i or original_q != recovered_q:
            symbol_errors += 1
    
    ser = symbol_errors / min_symbols
    
    # Error Vector Magnitude (EVM)
    evm = np.sqrt(np.mean(np.abs(original_symbols[:min_symbols] - recovered_symbols[:min_symbols])**2))
    
    print("\n=== ПРОИЗВОДИТЕЛЬНОСТЬ СИСТЕМЫ ===")
    print(f"Bit Error Rate (BER): {ber:.6f} ({bit_errors} ошибок из {min_len} битов)")
    print(f"Symbol Error Rate (SER): {ser:.6f} ({symbol_errors} ошибок из {min_symbols} символов)")
    print(f"Error Vector Magnitude (EVM): {evm:.6f}")
    
    return ber, ser, evm, bit_errors, symbol_errors

# Восстановление битов
recovered_bits, i_decisions, q_decisions = symbols_to_bits(recovered_symbols)

# Расчет характеристик
ber, ser, evm, bit_errors, symbol_errors = calculate_performance(
    source_bits, recovered_bits, symbols, recovered_symbols)

# Детальное сравнение данных
fig = plt.figure(figsize=(16, 12))
gs = gridspec.GridSpec(3, 2)

# Сравнение всех битов
ax1 = plt.subplot(gs[0, :])
bit_indices = np.arange(min(100, len(source_bits), len(recovered_bits)))
ax1.stem(bit_indices, source_bits[:len(bit_indices)], 
         linefmt='b-', markerfmt='bo', basefmt=' ', label='Переданные биты')
ax1.stem(bit_indices, recovered_bits[:len(bit_indices)], 
         linefmt='r--', markerfmt='rx', basefmt=' ', label='Принятые биты')

# Подсветка ошибок
error_indices = bit_indices[source_bits[:len(bit_indices)] != recovered_bits[:len(bit_indices)]]
ax1.plot(error_indices, np.ones_like(error_indices)*1.1, 'rv', 
         markersize=10, label='Ошибки', alpha=0.7)

ax1.set_title('ДЕТАЛЬНОЕ СРАВНЕНИЕ ПЕРЕДАННЫХ И ПРИНЯТЫХ БИТОВ', fontsize=14, fontweight='bold')
ax1.set_xlabel('Индекс бита')
ax1.set_ylabel('Значение бита')
ax1.legend()
ax1.grid(True)
ax1.set_ylim(-0.1, 1.4)

# Таблица ошибок
ax2 = plt.subplot(gs[1, 0])
ax2.axis('off')
error_table_data = [
    ["Параметр", "Значение"],
    ["Всего битов", f"{min(len(source_bits), len(recovered_bits))}"],
    ["Ошибок битов", f"{bit_errors}"],
    ["BER", f"{ber:.6f}"],
    ["Ошибок символов", f"{symbol_errors}"],
    ["SER", f"{ser:.6f}"],
    ["EVM", f"{evm:.6f}"]
]

table = ax2.table(cellText=error_table_data, 
                 cellLoc='center', 
                 loc='center',
                 bbox=[0.1, 0.1, 0.8, 0.8])
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1, 2)
ax2.set_title('СТАТИСТИКА ОШИБОК', fontweight='bold')

# Сравнение созвездий с ошибками
ax3 = plt.subplot(gs[1, 1])
# Создаем маски для правильных и ошибочных символов
correct_mask = np.zeros(len(recovered_symbols), dtype=bool)
for i in range(min(len(recovered_symbols), len(source_bits)//2)):
    original_i = 1 if np.real(symbols[i]) > 0 else 0
    original_q = 1 if np.imag(symbols[i]) > 0 else 0
    recovered_i = 1 if np.real(recovered_symbols[i]) > 0 else 0
    recovered_q = 1 if np.imag(recovered_symbols[i]) > 0 else 0
    
    correct_mask[i] = (original_i == recovered_i) and (original_q == recovered_q)

correct_symbols = recovered_symbols[correct_mask]
error_symbols = recovered_symbols[~correct_mask]

ax3.scatter(np.real(correct_symbols), np.imag(correct_symbols), 
           c='green', alpha=0.6, s=50, label='Правильные символы')
ax3.scatter(np.real(error_symbols), np.imag(error_symbols), 
           c='red', alpha=0.8, s=80, label='Ошибочные символы', marker='x')

# Исходное созвездие
for point, color, label in zip(constellation_points, colors, labels):
    ax3.scatter(np.real(point), np.imag(point), c=color, s=200, alpha=0.3)
    ax3.text(np.real(point)+0.1, np.imag(point)+0.1, label, fontsize=12)

ax3.set_title('Созвездие с выделением ошибок')
ax3.set_xlabel('I компонента')
ax3.set_ylabel('Q компонента')
ax3.legend()
ax3.grid(True)
ax3.axis('equal')

# Распределение ошибок по времени
ax4 = plt.subplot(gs[2, :])
symbol_time_indices = np.arange(len(recovered_symbols))
error_mask = ~correct_mask

ax4.plot(symbol_time_indices, np.abs(recovered_symbols), 'b-', 
         alpha=0.5, label='Амплитуда символов')
ax4.plot(symbol_time_indices[error_mask], np.abs(recovered_symbols[error_mask]), 'ro',
         markersize=8, label='Ошибочные символы')

ax4.set_title('РАСПРЕДЕЛЕНИЕ ОШИБОК ВО ВРЕМЕНИ', fontweight='bold')
ax4.set_xlabel('Индекс символа')
ax4.set_ylabel('Амплитуда символа')
ax4.legend()
ax4.grid(True)

plt.tight_layout()
plt.show()

# Итоговый вывод
print("\n" + "="*60)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("="*60)
print(f"✓ Успешно передано и восстановлено {min(len(source_bits), len(recovered_bits))} битов")
print(f"✓ Количество битовых ошибок: {bit_errors}")
print(f"✓ Bit Error Rate: {ber:.2%}")
print(f"✓ Качество связи: {'ОТЛИЧНО' if ber < 0.01 else 'ХОРОШО' if ber < 0.05 else 'УДОВЛЕТВОРИТЕЛЬНО'}")
print("="*60)